In [17]:
import os
import json
import pandas as pd
from pycocotools.coco import COCO

In [18]:
# Define file paths
csv_file = "../data/csv/depth-anything-v2-updated-with-annotations-v2.csv"    # Your CSV file
json_file = '../data/depth-anything-v2-set2/annotations/instances_default.json'  # Your JSON file

# Load the CSV file into a DataFrame
csv_data = pd.read_csv(csv_file)

# Load the COCO annotations
coco = COCO(json_file)


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [19]:
# Create a mapping from (image_id, annotation_id) to annotation data
annotations = coco.loadAnns(coco.getAnnIds())
annotation_mapping = {}

for ann in annotations:
    key = (ann['image_id'], ann['id'])  # Tuple of image_id and annotation_id
    annotation_mapping[key] = ann  # Store the entire annotation


In [20]:
# Ensure the new columns exist
csv_data['segmentation'] = None
csv_data['bbox'] = None

# Iterate over each row in the CSV DataFrame
for index, row in csv_data.iterrows():
    image_name = row['image_name']
    annotation_id = row['annotation_id']
    image_id = row['image_id']
    
    # Convert to integer if necessary
    if isinstance(annotation_id, float):
        annotation_id = int(annotation_id)
    if isinstance(image_id, float):
        image_id = int(image_id)
    
    # Create the key for annotation mapping
    key = (image_id, annotation_id)
    
    # Check if the key exists in the annotation mapping
    if key in annotation_mapping:
        ann = annotation_mapping[key]
        # Assign segmentation and bbox to the DataFrame
        csv_data.at[index, 'segmentation'] = json.dumps(ann['segmentation'])
        csv_data.at[index, 'bbox'] = ann['bbox']
    else:
        print(f"Annotation not found for image_name: {image_name}, image_id: {image_id}, annotation_id: {annotation_id}")


In [21]:
# Save the updated DataFrame to a new CSV file
output_csv_file = "../data/csv/depth-anything-v2-updated-with-annotations-v2.csv"
csv_data.to_csv(output_csv_file, index=False)

print(f"Updated CSV file saved to {output_csv_file}")


Updated CSV file saved to ../data/csv/depth-anything-v2-updated-with-annotations-v2.csv
